In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("FINAL_SALES_FILE_DATATHON-Copy1.csv")
df.drop(["Unnamed: 0"], 1, inplace = True)
df.drop_duplicates("SALESFILE_ID", inplace= True)
df.SF_CREATE_DATE = pd.to_datetime("2021/06/01") - pd.to_datetime(df.SF_CREATE_DATE)
df.SF_CREATE_DATE = df.SF_CREATE_DATE.dt.days
df = df.sort_values("SF_CREATE_DATE", ascending = False)
df.loc[df.SALESFILE_ID.isnull(), "SALESFILE_ID"] = 0

In [ ]:
final_customer = pd.read_csv("FINAL_CUSTOMER_DATATHON-Copy1.csv")
final_customer.drop(["Unnamed: 0"] ,1, inplace = True)
df = pd.merge(final_customer, df, on = "CUSTOMER_ID", how = "outer")
del final_customer
df.drop_duplicates(inplace =True)
df.loc[df.GENDER_ID == 0, "GENDER_ID"] = 100
df.drop(["GENDER"], 1, inplace = True)
df.drop(["CUSTOMER_ID"], 1, inplace = True)
df.drop_duplicates(inplace = True)
import numpy as np
f = lambda x: x.median() if np.issubdtype(x.dtype, np.number) else next(iter(x.mode()), np.nan)
df.MARITAL_STATUS.fillna(df.groupby('BASE_CUSTOMER_ID')["MARITAL_STATUS"].transform(f), inplace = True)
df.drop(["MARITAL_STATUS_ID"], 1, inplace = True)
df.BIRTH_DATE.fillna(df.groupby('BASE_CUSTOMER_ID')["BIRTH_DATE"].transform(f), inplace = True)

In [ ]:
import datetime   # 0 indicates 2021/05/31 which is fine
def serial_date_to_string(srl_no):
    new_date = datetime.datetime(2021,6,1,0,0) - datetime.timedelta(srl_no )
    return new_date.strftime("%Y-%m-%d")
df.SF_CREATE_DATE.fillna(-30, inplace = True)
df.SF_CREATE_DATE = df.SF_CREATE_DATE.transform(serial_date_to_string)

df["Age"] = pd.to_datetime(df["SF_CREATE_DATE"]).dt.year - df["BIRTH_DATE"]

In [ ]:
df.SF_CREATE_DATE = pd.to_datetime("2021/06/01") - pd.to_datetime(df.SF_CREATE_DATE)
df.SF_CREATE_DATE = df.SF_CREATE_DATE.dt.days
df.drop(["BIRTH_DATE"], 1, inplace = True)

df.loc[df.Age>85, "Age"] = np.nan
df.loc[df.Age< 15, "Age"] = np.nan
df.loc[(df.OCCUPATION == "Emekli") & (df.Age <40),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION == "Öğrenci") & (df.Age > 24), "OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION != "Öğrenci") & (df.Age <20),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor"
df.loc[(df.OCCUPATION != "Emekli") & (df.Age >64),"OCCUPATION"] = "Emekli"

df["occupation_fill"] = df.groupby("BASE_CUSTOMER_ID", sort=False)['OCCUPATION'].apply(lambda x: x.bfill().ffill())
df.drop(["OCCUPATION"], 1, inplace = True)
df.rename(columns = {"occupation_fill": "OCCUPATION"}, inplace = True)
df.Age.fillna(df.groupby("OCCUPATION")["Age"].mean(), inplace = True)
df.loc[df.MARITAL_STATUS.isnull(), "MARITAL_STATUS"] = "Diğer"
df.loc[(df.MARITAL_STATUS == "Bekar") & (df.Age.isnull()), "Age"] = df.loc[df.MARITAL_STATUS == "Bekar","Age"].mean()
df.loc[(df.MARITAL_STATUS == "Diğer") & (df.Age.isnull()), "Age"] = df.loc[df.MARITAL_STATUS == "Diğer","Age"].mean()
df.loc[(df.MARITAL_STATUS == "Evli") & (df.Age.isnull()), "Age"] = df.loc[df.MARITAL_STATUS == "Evli","Age"].mean()

df.loc[(df.OCCUPATION == "Emekli") & (df.Age <40),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION == "Öğrenci") & (df.Age > 24), "OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION != "Öğrenci") & (df.Age <20),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor"
df.loc[(df.OCCUPATION != "Emekli") & (df.Age >64),"OCCUPATION"] = "Emekli"

df["address_fill"] = df.groupby("BASE_CUSTOMER_ID",
                                sort=False)['FK_ADDRESS_COMMUNICATION_CITY'].apply(lambda x: x.bfill().ffill())
s = df.address_fill.value_counts(normalize=True)
missing = df['address_fill'].isnull()
df.loc[missing,'address_fill'] = np.random.choice(s.index, size=len(df[missing]),p=s.values)
df.drop(["FK_ADDRESS_COMMUNICATION_CITY"], 1, inplace =True)
df.drop_duplicates(inplace = True)

In [ ]:
df.REQ_BRAND_CODE.fillna(df.groupby('BASE_CUSTOMER_ID')["REQ_BRAND_CODE"].transform(f), inplace = True)
df.REQ_TOPMODEL_CODE.fillna(df.groupby('BASE_CUSTOMER_ID')["REQ_TOPMODEL_CODE"].transform(f), inplace = True)

df.to_csv("TEST_ORAN_FINAL_SALES.csv", index =False)

INCLUDED PART

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("FINAL_SALES_FILE_DATATHON-Copy1.csv")
df.drop(["Unnamed: 0"], 1, inplace = True)
df.drop_duplicates("SALESFILE_ID", inplace= True)
df.SF_CREATE_DATE = pd.to_datetime("2021/06/01") - pd.to_datetime(df.SF_CREATE_DATE)
df.SF_CREATE_DATE = df.SF_CREATE_DATE.dt.days
df.SF_CREATE_DATE = df.SF_CREATE_DATE - 92
df.loc[df.SALESFILE_ID > 0, "SALESFILE_ID"] = 1
df.loc[df.SF_CREATE_DATE < 0, "SALESFILE_ID"] = 2
df = df.sort_values("SALESFILE_ID")
df.loc[df.SALESFILE_ID.isnull(), "SALESFILE_ID"] = 0

final_customer = pd.read_csv("FINAL_CUSTOMER_DATATHON-Copy1.csv")
final_customer.drop(["Unnamed: 0"] ,1, inplace = True)
df = pd.merge(final_customer, df, on = "CUSTOMER_ID", how = "outer")
del final_customer
df.drop_duplicates(inplace =True)
df.loc[df.SALESFILE_ID.isnull(), "SALESFILE_ID"] = 0

df.loc[df.GENDER_ID == 0, "GENDER_ID"] = 100
df.drop(["GENDER"], 1, inplace = True)
df.drop(["CUSTOMER_ID"], 1, inplace = True)
df.drop_duplicates(inplace = True)
import numpy as np
f = lambda x: x.median() if np.issubdtype(x.dtype, np.number) else next(iter(x.mode()), np.nan)
df.MARITAL_STATUS.fillna(df.groupby('BASE_CUSTOMER_ID')["MARITAL_STATUS"].transform(f), inplace = True)
df.drop(["MARITAL_STATUS_ID"], 1, inplace = True)
df.BIRTH_DATE.fillna(df.groupby('BASE_CUSTOMER_ID')["BIRTH_DATE"].transform(f), inplace = True)
df.SF_CREATE_DATE.fillna(0, inplace = True)
df.SF_CREATE_DATE = df.SF_CREATE_DATE.transform(serial_date_to_string)

df["Age"] = pd.to_datetime(df["SF_CREATE_DATE"]).dt.year - df["BIRTH_DATE"]

In [ ]:
df.SF_CREATE_DATE = pd.to_datetime("2021/03/01") - pd.to_datetime(df.SF_CREATE_DATE)
df.SF_CREATE_DATE = df.SF_CREATE_DATE.dt.days
df.drop(["BIRTH_DATE"], 1, inplace = True)

df.loc[df.Age>85, "Age"] = np.nan
df.loc[df.Age< 15, "Age"] = np.nan
df.loc[(df.OCCUPATION == "Emekli") & (df.Age <40),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION == "Öğrenci") & (df.Age > 24), "OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION != "Öğrenci") & (df.Age <20),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor"
df.loc[(df.OCCUPATION != "Emekli") & (df.Age >64),"OCCUPATION"] = "Emekli"

df["occupation_fill"] = df.groupby("BASE_CUSTOMER_ID", sort=False)['OCCUPATION'].apply(lambda x: x.bfill().ffill())
df.drop(["OCCUPATION"], 1, inplace = True)
df.rename(columns = {"occupation_fill": "OCCUPATION"}, inplace = True)
df.Age.fillna(df.groupby("OCCUPATION")["Age"].mean(), inplace = True)
df.loc[df.MARITAL_STATUS.isnull(), "MARITAL_STATUS"] = "Diğer"
df.loc[(df.MARITAL_STATUS == "Bekar") & (df.Age.isnull()), "Age"] = df.loc[df.MARITAL_STATUS == "Bekar","Age"].mean()
df.loc[(df.MARITAL_STATUS == "Diğer") & (df.Age.isnull()), "Age"] = df.loc[df.MARITAL_STATUS == "Diğer","Age"].mean()
df.loc[(df.MARITAL_STATUS == "Evli") & (df.Age.isnull()), "Age"] = df.loc[df.MARITAL_STATUS == "Evli","Age"].mean()

df.loc[(df.OCCUPATION == "Emekli") & (df.Age <40),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION == "Öğrenci") & (df.Age > 24), "OCCUPATION"] = "Çalışmıyor/Bilinmiyor" 
df.loc[(df.OCCUPATION != "Öğrenci") & (df.Age <20),"OCCUPATION"] = "Çalışmıyor/Bilinmiyor"
df.loc[(df.OCCUPATION != "Emekli") & (df.Age >64),"OCCUPATION"] = "Emekli"

df["address_fill"] = df.groupby("BASE_CUSTOMER_ID",
                                sort=False)['FK_ADDRESS_COMMUNICATION_CITY'].apply(lambda x: x.bfill().ffill())
s = df.address_fill.value_counts(normalize=True)
missing = df['address_fill'].isnull()
df.loc[missing,'address_fill'] = np.random.choice(s.index, size=len(df[missing]),p=s.values)
df.drop(["FK_ADDRESS_COMMUNICATION_CITY"], 1, inplace =True)
df.drop_duplicates(inplace = True)

In [ ]:
df.REQ_BRAND_CODE.fillna(df.groupby('BASE_CUSTOMER_ID')["REQ_BRAND_CODE"].transform(f), inplace = True)
df.REQ_TOPMODEL_CODE.fillna(df.groupby('BASE_CUSTOMER_ID')["REQ_TOPMODEL_CODE"].transform(f), inplace = True)
df.to_csv("TRAIN_ORAN_final_sales.csv", index = False)